5 6 2 2 1 1 3 2

In [29]:
w, h, f, c, x1, y1, x2, y2 = map(int, input().split())

after_x = max(w - f, f)
after_y = h // (c + 1)
mul = c + 1

total = 0
total += mul * max(0, x2 - max(x1, min(f, w - f))) * (y2 - y1)
total += 2 * mul * (y2 - y1) * max(0, min(x2, min(f, w - f)) - x1)
print(w * h - total)

 4 8 3 0 1 1 3 2


30


In [22]:
2 * mul * (y2 - y1) * max(0, min(x2, min(f, w - f)) - x1)

6

In [4]:
mul * (y2 - y1) * max(0, max(after_x - f, 2 * f - w) - x1)

0

In [2]:
print(after_x, after_y)

3 2


## 8-2. 데이터 전처리

- 데이터 추출
    - df1.head() : 상위 n개 데이터 추출
    - df1.tail() : 하위 n개 데이터 추출
    - df1.iloc[n:m] : n번째 행 부터 m번째 행 까지 추출
    - df1['col'] : 특정 col을 추출 (Series 형태로 추출)
    - df1[['col1', 'col2']] : 한 개 이상의 col을 데이터 프레임형태로 추출

- 데이터 정렬
    - df1.sort_values(by = col, ascending = True) : 특정 col열을 오름차순/ 내림차순으로 정렬

- 데이터 필터 (층별화 - 특정 범주항목 별, 데이터를 나누어 분석)
    - df1.loc[cond1] : cond1 (특정 조건을 만족하는, 비교구문에 있어 True 출력되는 행) 에 대해 데이터를 출력
    - df1.loc[cond1, new_col] = 'new_data' : cond1에 대해 new_data를 new_col으로 선언 (특정 조건에 대해 새로운 데이터를 파생변수로 선언)

- 데이터 요약
    - df1. pivot_table(index = 구분하고자 하는 항목, values = 계산하고자 하는 항목, aggfunc = 통계량)
    - 특정 항목을 기준으로 특정 값을 원하는 통계량에 대해 계산
    - columns = : 구분하고자 하는 항목을 column으로 생성
    - margins = 특정 통계량을 행/열 단위로 계산
    - fill_values : 요약을 수행할 때 발생한 결측값을 특정 값으로 대치

- 파생면수 생성
    - df1.apply(function) : 특정 column의 각 row를 사용자가 만든(정의한) function에 넣어 계산
 
- 날짜 데이터 처리
    - pd.to_datetime(df1[col]) : 특정 날짜 형식(col)의 자료를 날짜형태 타입으로 변환
    - df1[col_dt].dt.year : 연도 추출
    - df1[col_dt].dt.month : 월 추출
    - df1[col_dt].dt.day : 일자 추출
    - df1[col_dt].dt.day_name() : 요일 추출
    - df1[col_dt].dt.isocalender().week : 주차 추출 (work week)

- 결측값 처리
- 데이터 재구조화 및 병합

**결측값 처리**

- 결측값 (missing value -> Nan, Not A Number)
- 결측값 제거 : 결측값이 존재하는 행을 제거
- 결측값 대치 : 결측값을 다른 값으로 채워 넣는 방법
    - 단순대치 : 평균이나 중강값과 같은 해당col을 대표할 수 있는 값으로 대치
    - 알고리즘 대치 : 머신러닝 알고리즘들을 활용해 해당 결측값과 유사한 값으로 대치
    - 보간법 : 특정 시간 간격이나 서열 간격에 맞춰 데이터를 생성해 대치

In [5]:
import pandas as pd

In [7]:
df1 = pd.read_csv('data/01_Data.csv') # 데이터 호출
print(df1.shape) # 데이터의 행, 열 개수 확인
df1.head() # 데이터 확인

(51301, 20)


,Unnamed: 0,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,Amount_Month,Customer_Type,Age,Address1,State,Overdue_count,Overdue_Type,Gender,Bank,Credit_Score
0,0,1,66758234,렌탈,일반계약,영업방판,2022-05-09,60,CMS,SLX-1,96900,개인,42.0,경기도,계약확정,0,없음,여자,새마을금고,496.0
1,1,2,66755948,렌탈,교체계약,영업방판,2023-02-23,60,카드이체,SLX-1,102900,개인,39.0,경기도,계약확정,0,없음,남자,현대카드,876.0
2,2,3,66756657,렌탈,일반계약,홈쇼핑/방송,2022-03-03,60,CMS,SLX-1,96900,개인,48.0,경기도,계약확정,0,없음,여자,우리은행,512.0
3,3,4,66423450,멤버십,멤버십3유형,재계약,2022-05-16,12,CMS,SLX-1,66900,개인,39.0,경기도,계약확정,0,없음,남자,농협회원조합,777.0
4,4,5,66423204,멤버십,멤버십3유형,재계약,2022-05-13,12,CMS,SLX-1,66900,개인,60.0,경기도,기간만료,12,있음,남자,농협회원조합,561.0


In [11]:
# 결측값 확인
df1.info() # Non-Null count : 결측값이 아닌 데이터 수

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51301 entries, 0 to 51300
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     51301 non-null  int64  
 1   Index          51301 non-null  int64  
 2   Member_ID      51301 non-null  int64  
 3   Sales_Type     51301 non-null  object 
 4   Contract_Type  51301 non-null  object 
 5   Channel        51301 non-null  object 
 6   Datetime       51301 non-null  object 
 7   Term           51301 non-null  int64  
 8   Payment_Type   51301 non-null  object 
 9   Product_Type   51301 non-null  object 
 10  Amount_Month   51301 non-null  int64  
 11  Customer_Type  51299 non-null  object 
 12  Age            44329 non-null  float64
 13  Address1       51299 non-null  object 
 14  State          51301 non-null  object 
 15  Overdue_count  51301 non-null  int64  
 16  Overdue_Type   51301 non-null  object 
 17  Gender         51301 non-null  object 
 18  Bank  

In [13]:
df1.isnull().sum() # 특정 데이터가 결측값인지 아닌지 출력 -> sum함수를 사용하면 True값의 개수를 column별 합을 계산

Unnamed: 0          0
Index               0
Member_ID           0
Sales_Type          0
Contract_Type       0
Channel             0
Datetime            0
Term                0
Payment_Type        0
Product_Type        0
Amount_Month        0
Customer_Type       2
Age              6972
Address1            2
State               0
Overdue_count       0
Overdue_Type        0
Gender              0
Bank             2759
Credit_Score     8841
dtype: int64

In [16]:
# 결측값을 제거 (특정 행에 결측값이 존재하면, 해당 행의 정보를 삭제)
df1_clean = df1.dropna()
df1_clean.shape

(40597, 20)

In [17]:
# 결측값이 많은 열을 제거한 뒤 결측값을 제거하여 분석
# drop(): 특정 행 또는 열을 삭제
df1_clean2 = df1.drop(columns = ['Age', 'Bank', 'Credit_Score']).dropna()
df1_clean2.shape

(51299, 17)

In [18]:
# 결측값 대치 (단순대치)
df1['Credit_Score'].tail(10)

51291      NaN
51292      NaN
51293    978.0
51294      NaN
51295    911.0
51296      NaN
51297    574.0
51298    972.0
51299    888.0
51300    538.0
Name: Credit_Score, dtype: float64

In [20]:
# 결측값을 0으로 대치하여 처리
df1['Credit_Score_clean'] = df1['Credit_Score'].fillna(0)
df1['Credit_Score_clean'].tail(10)

51291      0.0
51292      0.0
51293    978.0
51294      0.0
51295    911.0
51296      0.0
51297    574.0
51298    972.0
51299    888.0
51300    538.0
Name: Credit_Score_clean, dtype: float64

원시자료 설명서(DB설계서) -> 결측값이 다른 숫자값으로 대치되었는지 확인할 수 있다.

In [31]:
# 결측값을 평균으로 대치하여 처리
df1['Credit_Score_clean2'] = df1['Credit_Score'].fillna(df1['Credit_Score'].mean())
df1['Credit_Score_clean2'].tail(10)

51291    826.838342
51292    826.838342
51293    978.000000
51294    826.838342
51295    911.000000
51296    826.838342
51297    574.000000
51298    972.000000
51299    888.000000
51300    538.000000
Name: Credit_Score_clean2, dtype: float64

## 8-3. 기술통계량 확인

- 데이터의 특성을 알기 위해 기본적으로 확인하는 통계량
- 연속형 :
    - 대표값 (평균, 중앙값): 해당 연속형 자료를 대표하는 수치
        - 평균의 경우, 이상치(outlier)에 의해 매우 높거나 낮게 계산될 수 있으므로, 반드시 중위수(중앙값)와 같이 확인
        - A : 1, 3, 2, 5, 4 -> 평균 : 3
        - A : 1, 2, 3, 4, 5 -> 중앙값 : 3
        - B : 1, 3, 2, 4, 10000 -> 평균 : 2002
        - B : 1, 2, 3, 4, 10000 -> 중앙값 : 3
 
    - 산포 (표준편차) : 해당 연속형자료의 데이터들이 대표값으로부터 얼마나 떨어져 있는지 나타내는 통계량
        - 산포는 데이터가 얼마나 대표값에 대해 정확한가를 나타내는 척도
        - 데이터가 많이 수집 될 수록, 대표값에 대해 모여있는 분포를 나타낸(중심극한의 정리)
        - 데이터가 많아지면, 산포가 줄어들 가능성이 있음

    - 분포의 모양 : 해당 연속형 자료의 분포의 모양, 시각화
        - 정규분포, 정규분포가 아닌 데이터
        - 분포의 모양에 따라 가설검정 및 통계적 분석 방법이 달라질 수 있음
        - 데이터 수집 관점

- 범주형 : 항목과 빈도수 확인 (Unique / Frequency), 비율

In [33]:
# 월 랜탈비용 (Amount_Month)과 계약 기간(월) (Term)을 곱하여, 총 계약금을 계산해, 통계량을 확인
df1['총계약금'] = df1['Amount_Month'] * df1['Term']

In [35]:
print(df1['총계약금'].mean()) # 해당 시리즈의 평균
print(df1['총계약금'].median()) # 해당 시리즈의 중앙값

5315393.8288922245
5814000.0


In [38]:
# 산포
# 표준편차: 평균으로부터 데이터가 얼마나 떨어져 있는가 (평균 : 이상치의 영향을 받는 값)
# 범위 : 최댓값과 최솟값의 차이
# 사분범위(IQR) : 전체 데이터의 50% 모여있는 구간 (25% 와  75%사이)
print(df1['총계약금'].std()) # 표준편차
print(df1['총계약금'].quantile(0.25)) # 25% 1사분범위 값
print(df1['총계약금'].quantile(0.75)) # 75% 3사분범위 값
print(df1['총계약금'].quantile(0.75) - df1['총계약금'].quantile(0.25)) # IQR

1455793.805881198
4914000.0
5814000.0
900000.0


In [40]:
df1.describe() # 모든 숫자 자료의 기술 통계량을 계싼

,Unnamed: 0,Index,Member_ID,Term,Amount_Month,Age,Overdue_count,Credit_Score,Credit_Score_clean,Credit_Score_clean2,총계약금
count,51301.000000,51301.000000,5.130100e+04,51301.000000,51301.000000,44329.000000,51301.000000,42460.000000,51301.000000,51301.000000,5.130100e+04
mean,25650.000000,25651.703612,6.266432e+07,55.639149,93994.974289,50.024093,0.161381,826.838342,684.344477,826.838342,5.315394e+06
std,14809.467417,14809.828628,1.216146e+07,12.009915,15304.263988,10.983877,1.122193,118.301113,330.302938,107.625462,1.455794e+06
min,0.000000,1.000000,2.568798e+07,12.000000,54603.000000,25.000000,0.000000,2.000000,0.000000,2.000000,6.624000e+05
25%,12825.000000,12826.000000,6.643148e+07,60.000000,81900.000000,42.000000,0.000000,772.000000,635.000000,800.000000,4.914000e+06
50%,25650.000000,25652.000000,6.676578e+07,60.000000,96900.000000,49.000000,0.000000,856.000000,828.000000,828.000000,5.814000e+06
75%,38475.000000,38477.000000,6.678160e+07,60.000000,98400.000000,57.000000,0.000000,902.000000,891.000000,891.000000,5.814000e+06
max,51300.000000,51302.000000,6.696986e+07,60.000000,215700.000000,102.000000,15.000000,1000.000000,1000.000000,1000.000000,1.006200e+07


In [42]:
# 범주형 (항목과 빈도수 확인)
df1['Product_Type'].unique() # 특정 범주형 항목에 있는 값을 확인

array(['SLX-1', 'SLX-3', 'SLX-2', 'ADV-5', '3DIR-AVD', 'ADVERA'],
      dtype=object)

In [45]:
df1['Product_Type'].value_counts() # 특정 범주형 항목의 빈도수 확인 -> Series 형태로 출력

Product_Type
SLX-1       39133
SLX-2        8996
SLX-3        2081
ADVERA        644
ADV-5         327
3DIR-AVD      120
Name: count, dtype: int64

In [47]:
df1['Product_Type'].value_counts().index # 제품군의 빈도수가 높은 순서대로 나열된 시리즈에서 각 항목의 순서를 arrary 형태로 출력

Index(['SLX-1', 'SLX-2', 'SLX-3', 'ADVERA', 'ADV-5', '3DIR-AVD'], dtype='object', name='Product_Type')

In [53]:
df1['Product_Type'].value_counts().head(3).index

Index(['SLX-1', 'SLX-2', 'SLX-3'], dtype='object', name='Product_Type')

In [56]:
# 제품군의 빈도수가 높은 3개 제품에 대해 총 계약금의 기술통계량을 확인하여 excel로 저장
# 제품군의 빈도수가 높은 상위 3개 제품을 추출해 변수로 선언
top_list = df1['Product_Type'].value_counts().head(3).index
# 해당 리스트에 있는 제품군에 해당하는 데이터를 추출
cond1 = df1['Product_Type'].isin(top_list)
df1.loc[cond1].describe()[['총계약금']]

,총계약금
count,5.021000e+04
mean,5.303260e+06
std,1.450604e+06
min,6.624000e+05
25%,4.914000e+06
50%,5.814000e+06
75%,5.814000e+06
max,1.006200e+07


In [58]:
# 범주형 데이터에 대한 통계량
df1.describe(include = 'object')
# unique : 해당 범주형 항목의 개수
# top : 최빈값, 가장 빈도수가 많은 항목
# freq : 최빈값의 개수, 가장 빈도수가 많은 항목의 수

,Sales_Type,Contract_Type,Channel,Datetime,Payment_Type,Product_Type,Customer_Type,Address1,State,Overdue_Type,Gender,Bank
count,51301,51301,51301,51301,51301,51301,51299,51299,51301,51301,51301,48542
unique,2,9,16,577,5,6,2,8,4,2,2,47
top,렌탈,프로모션계약,영업방판,2022-02-03,CMS,SLX-1,개인,경기도,계약확정,없음,여자,롯데카드
freq,46483,15811,23767,1167,32825,39133,46263,18353,50620,49110,35602,9516


In [59]:
df1.describe(include = 'all') # 모든 데이터 타입에 대한 기술통계량 확인

,Unnamed: 0,Index,Member_ID,Sales_Type,Contract_Type,Channel,Datetime,Term,Payment_Type,Product_Type,...,Address1,State,Overdue_count,Overdue_Type,Gender,Bank,Credit_Score,Credit_Score_clean,Credit_Score_clean2,총계약금
count,51301.000000,51301.000000,5.130100e+04,51301,51301,51301,51301,51301.000000,51301,51301,...,51299,51301,51301.000000,51301,51301,48542,42460.000000,51301.000000,51301.000000,5.130100e+04
unique,NaN,NaN,NaN,2,9,16,577,NaN,5,6,...,8,4,NaN,2,2,47,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,렌탈,프로모션계약,영업방판,2022-02-03,NaN,CMS,SLX-1,...,경기도,계약확정,NaN,없음,여자,롯데카드,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,46483,15811,23767,1167,NaN,32825,39133,...,18353,50620,NaN,49110,35602,9516,NaN,NaN,NaN,NaN
mean,25650.000000,25651.703612,6.266432e+07,NaN,NaN,NaN,NaN,55.639149,NaN,NaN,...,NaN,NaN,0.161381,NaN,NaN,NaN,826.838342,684.344477,826.838342,5.315394e+06
std,14809.467417,14809.828628,1.216146e+07,NaN,NaN,NaN,NaN,12.009915,NaN,NaN,...,NaN,NaN,1.122193,NaN,NaN,NaN,118.301113,330.302938,107.625462,1.455794e+06
min,0.000000,1.000000,2.568798e+07,NaN,NaN,NaN,NaN,12.000000,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,2.000000,0.000000,2.000000,6.624000e+05
25%,12825.000000,12826.000000,6.643148e+07,NaN,NaN,NaN,NaN,60.000000,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,772.000000,635.000000,800.000000,4.914000e+06
50%,25650.000000,25652.000000,6.676578e+07,NaN,NaN,NaN,NaN,60.000000,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,856.000000,828.000000,828.000000,5.814000e+06
75%,38475.000000,38477.000000,6.678160e+07,NaN,NaN,NaN,NaN,60.000000,NaN,NaN,...,NaN,NaN,0.000000,NaN,NaN,NaN,902.000000,891.000000,891.000000,5.814000e+06


**데이터 재구조화**

- E 커머스 데이터, 특정 생필품을 납품하는 업체가 온라인 쇼핌몰 플랫폼에 물픔을 얼만큼 납푼했는지에 대한 자료
- ERP(전사적 자원 관리, Enterprise Resource Planning)에서 발생한 데이터들이 갖는 유형 처리
- DB 매니저들이 사원들의 업무처리가 쉽도록, 엑셀에서 쉽게 사용할 수 있는 형태로 바꿔놓은 경우

In [61]:
# 데이터 재구조화
df2 = pd.read_excel('data/02_Data.xlsx', skiprows = 1) # skiprows : 데이터를 불러오는 단계에서 특정 index를 생략
df2.shape

(40, 17)

|제품명|홍길동|성춘향|이몽룡|허준|
|------|---|---|---|---|
|A|1000|500|200|400|
|B|1500|90|100|700|
|C|...||||
|D||||
|...||||


이거를 

|제품명|사원명|매출액|
|------|---|---|
|A|홍길동|1000|
|A|성춘향|500|
|A|이몽룡|200|
|A|허준|400|
|B|||
|B|||
|B|||
|...|...|...|

In [64]:
df2.columns # -> 날짜를 하나의 column으로 해야함

Index([            '카테고리명',             '자재그룹명',               '제품명',
       2019-12-01 00:00:00, 2019-12-02 00:00:00, 2019-12-03 00:00:00,
       2019-12-04 00:00:00, 2019-12-05 00:00:00, 2019-12-06 00:00:00,
       2020-12-01 00:00:00, 2020-12-02 00:00:00, 2020-12-03 00:00:00,
       2020-12-04 00:00:00, 2020-12-05 00:00:00, 2020-12-06 00:00:00,
                    '안전재고',                '분류'],
      dtype='object')

In [65]:
df3 = df2.melt(id_vars = ['카테고리명', '자재그룹명', '제품명', '안전재고', '분류'])
df3.shape

(480, 7)

In [66]:
# 항목 이름 변경
df4 = df3.rename(columns = {'variable':'날짜', 'value':'매출액'})
df4.head(2)

,카테고리명,자재그룹명,제품명,안전재고,분류,날짜,매출액
0,CHB,E1,A,0.0,E,2019-12-01 00:00:00,0.0
1,CHB,E1,B,0.0,E,2019-12-01 00:00:00,0.0


In [68]:
# 2019년 12월 1일~6일과 2020년도 12월 1일~6일
# 2019년 대비 2020년 매출액 비교, 제품명 별 매출 비교
df4['날짜_dt'] = pd.to_datetime(df4['날짜'])
df4['연도'] = df4['날짜_dt'].dt.year
df4['연도'].unique()

array([2019, 2020], dtype=int32)

In [69]:
df4.pivot_table(index = '연도', values = '매출액', aggfunc = 'sum')

,매출액
연도,
2019,2115606.2
2020,684328.7


In [70]:
df4.pivot_table(index = '제품명', columns = '연도', values = '매출액', aggfunc = 'sum')

연도,2019,2020
제품명,,
A,98.8,98.8
A1,4.4,4.4
A2,176235.6,62059.8
A3,10302.0,46.7
B,393.7,393.7
B2,15122.5,11726.7
B3,60456.1,16949.8
C,48772.3,10353.0
D,34337.0,6718.1


**데이터 병합**

- 경기 소재지 친환경 식료품을 납품하는 소매점 데이터
- 회원제로 운영되는 매장 데이터
- 고객의 기본 개인정보와 고객이 구매할 때 바코드에 찍힌 물품 정보(판매 정보)가 있는 데이터

In [73]:
df_member = pd.read_csv('data/03_store_member.csv')
print(df_member.shape)
print(df_member.head(2))

(4396, 6)
   회원번호  회원상태 성별 결혼유무               주소  연령
0  2101  정상회원  여   기혼  서울 강북구 미아동 134-  51
1  2102  정상회원  여  NaN   경기 용인시 수지구 동천동  71


In [74]:
df_product = pd.read_csv('data/03_store_product.csv')
print(df_product.shape)
print(df_product.head(2))

(735783, 11)
       공급일자      물품코드 물품대분류   물품중분류     물품소분류           물품명  구매수량  구매금액 구매매장  \
0  20220201  50142001    채소  잎/줄기채소       시금치     시금치(300g)   1.0  2000  매장3   
1  20220201  60302012   축산물     육가공  돈가스/분쇄가공  튀김옷돈가스(400g)   1.0  6300  매장3   

   반품_원거래일자    회원번호  
0       NaN  2101.0  
1       NaN  2101.0  


In [75]:
df_product.isnull().sum()

공급일자             0
물품코드             0
물품대분류            0
물품중분류            0
물품소분류            0
물품명              0
구매수량             0
구매금액             0
구매매장             0
반품_원거래일자    725446
회원번호            22
dtype: int64

In [79]:
from collections import defaultdict

dic_list = []
while True:
    word = input()
    if word == "-":
        break
    else:
        dic = defaultdict(int)
        for w in word:
            dic[w] += 1
        dic_list.append(dic)

puzzle = []
while True:
    word = input()
    if word == "#":
        break
    else:
        dic = defaultdict(int)
        for w in word:
            dic[w] += 1
        puzzle.append(dic)





 APPLE
 BANANA
 BANE
 -
 A
 #


[defaultdict(<class 'int'>, {'A': 1, 'P': 2, 'L': 1, 'E': 1}), defaultdict(<class 'int'>, {'B': 1, 'A': 3, 'N': 2}), defaultdict(<class 'int'>, {'B': 1, 'A': 1, 'N': 1, 'E': 1})]


In [77]:
# 어떤 고객이 얼마나 매장에 많이 방문하고, 많은 금액을 사용하는지?
# reset_index(): index값을 0번부터 다시 부여
p1 = df_product.pivot_table(index = '회원번호', values = ['구매금액', '구매수량'], aggfunc = 'sum').reset_index()
p1.shape

(4285, 3)

In [83]:
# key colun (회원번호)에 공통된 값만 가져와 병합
m1 = pd.merge(df_member, p1, on = '회원번호', how = 'inner')
# how에는 inner, outer, left, right
# on : key column
print(m1.shape)
print(m1.head(2))

(4285, 8)
   회원번호  회원상태 성별 결혼유무               주소  연령     구매금액   구매수량
0  2101  정상회원  여   기혼  서울 강북구 미아동 134-  51  1991230  382.0
1  2102  정상회원  여  NaN   경기 용인시 수지구 동천동  71  2095860  472.5


In [88]:
# 한명의 고객 별로 언제 방문했는가를 1값으로 식별하기 위해, 회원-날짜-1 계산
df_product['Label'] = 1
p2 = df_product.pivot_table(index = ['회원번호', '공급일자'], values = 'Label', aggfunc = 'min').reset_index()
p3 = p2.pivot_table(index = '회원번호', values = 'Label', aggfunc = 'sum').reset_index()
print(p3.shape)
print(p3.head(2))

(4285, 2)
     회원번호  Label
0  2101.0     86
1  2102.0     79


In [92]:
pd.merge(m1, p3, on = '회원번호', how = 'inner')
df5 = df4.rename(columns = {'구매금액':'총구매금액', '구매수량': '총구매수량', "label":'총방문횟수'})
print(df5.shape)
print(df5.head())

(480, 9)
  카테고리명 자재그룹명 제품명     안전재고 분류                   날짜      매출액      날짜_dt    연도
0   CHB    E1   A      0.0  E  2019-12-01 00:00:00      0.0 2019-12-01  2019
1   CHB    E1   B      0.0  E  2019-12-01 00:00:00      0.0 2019-12-01  2019
2   CHB    E1   C      0.0  A  2019-12-01 00:00:00   9474.3 2019-12-01  2019
3   CHB    E1   D      0.0  A  2019-12-01 00:00:00   7702.6 2019-12-01  2019
4   CHB    E1   E  39593.3  A  2019-12-01 00:00:00  56309.6 2019-12-01  2019
